In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import expit
from tqdm import tqdm

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

from imblearn.under_sampling import RandomUnderSampler

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test-public.csv")
test["Pair"] = list(zip(test.Source, test.Sink))
#train_data = pd.read_csv("train_data.csv")
#train_data
weak_ties = pd.read_csv("weak_ties.csv")

In [4]:
train_data = pd.merge(train, weak_ties, how="left", on=["Pair"])
train_data

,Pair,Source,Sink,NCA,Exist,CN,AA,RA,JC,PA,...,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20,Unnamed: 0,WCN,WAA,WRA
0,"(0, 356)",0,356,14,1,7,2.899858,0.628968,0.700000,72,...,-0.398342,1.080715,-0.773911,-1.598859,0.635909,-0.709417,0,83.041696,20.598592,1.522746
1,"(0, 1236)",0,1236,14,1,6,2.471649,0.531746,0.428571,96,...,-0.223878,0.998884,-0.651569,-1.617872,0.695837,-0.712304,1,75.798683,18.822305,1.384504
2,"(356, 1236)",356,1236,14,1,7,2.812086,0.587302,0.500000,108,...,-0.200811,1.017463,-0.606597,-1.639789,0.612917,-0.744530,2,87.009568,21.650570,1.604475
3,"(0, 1655)",0,1655,9,1,7,2.976054,0.668651,0.466667,112,...,-0.570535,0.916353,-0.617127,-1.465398,0.716408,-0.587510,3,88.275209,22.088079,1.682613
4,"(0, 1797)",0,1797,4,1,7,2.899858,0.628968,0.700000,72,...,-0.440440,1.122925,-0.803164,-1.635018,0.791342,-0.652107,4,81.098878,20.322190,1.561990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7093258,"(4089, 4091)",4089,4091,0,0,0,0.000000,0.000000,0.000000,35,...,-0.301712,0.314469,0.645251,-0.759133,-0.681610,-1.176053,7093258,0.000000,0.000000,0.000000
7093259,"(4089, 4092)",4089,4092,0,0,0,0.000000,0.000000,0.000000,25,...,-0.828324,0.048068,-0.529535,-1.272614,-0.734793,-0.223739,7093259,0.000000,0.000000,0.000000
7093260,"(4090, 4091)",4090,4091,0,0,0,0.000000,0.000000,0.000000,42,...,-0.135803,0.239426,0.419526,-0.902752,-0.551571,-1.232052,7093260,0.000000,0.000000,0.000000
7093261,"(4090, 4092)",4090,4092,0,0,0,0.000000,0.000000,0.000000,30,...,-0.662415,-0.026975,-0.755260,-1.416234,-0.604754,-0.279738,7093261,0.000000,0.000000,0.000000


In [5]:
y = train_data.Exist
X = train_data.drop(columns=["Pair", "Source", "Sink", "NCA", "Exist", "CN", "AA", "RA", "Unnamed: 0"])
result = train_data[["Pair", 'Exist']]

In [9]:
# balance the dataset by undersampling
rus = RandomUnderSampler(sampling_strategy=0.2)
X_us, y_us= rus.fit_resample(X, y)
type(y_us)

pandas.core.series.Series

In [8]:
pd.array(result)

KeyboardInterrupt: 

In [42]:
from sklearn.linear_model import LogisticRegression

# Code adapted from Worksheet04 to calculate the coefficients
clf = LogisticRegression(penalty='none')
clf.fit(X_us, y_us)
w_sklearn = np.r_[clf.intercept_, clf.coef_.squeeze()]
print("Weights according to scikit-learn: {}".format(w_sklearn))

Weights according to scikit-learn: [-1.40733554e+00  1.14791542e+00 -8.30585682e-04  7.03201566e-01
  1.08602488e-03  1.43446187e-03  2.41315890e-02  2.23805529e-01
 -7.48699523e-02 -2.25082489e-01  3.31661881e-01  8.42094371e-02
 -9.77206273e-02 -6.55516506e-01  1.46836316e-01 -8.64054373e-02
 -4.82962051e-01  2.17577310e-01  1.23126909e-01  2.66915584e-02
 -3.13419666e-01 -2.07579934e-01 -2.97884074e-01  2.41598051e-01
  2.62770584e-01  4.46921569e-01 -8.63257312e-01  5.66456701e+00
  3.21981329e+00]


/Users/abigail/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [56]:
X.insert(0,'Intercept',1)
X.insert(0,'Pair',train.Pair)
#X.insert(0,'Exist',train.Exist)

In [57]:
X['Pair'] = X['Pair'].astype(str)
test['Pair'] = test['Pair'].astype(str)
result['Pair'] = result['Pair'].astype(str)
#test = test.merge(X, how='left', on='Pair')
test = test.merge(result, how='left', on='Pair')
test


/Users/abigail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Id,Source,Sink,Pair,Intercept_x,JC_x,PA_x,KI_x,PR_s1_x,PR_s2_x,...,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20,WCN,WAA,WRA,Exist_y
0,1,0,2917,"(0, 2917)",1.0,0.000000,56.0,-0.000552,0.000220,0.000185,...,0.123224,0.987519,-0.487011,-1.497914,0.483864,-0.793305,0.000000,0.000000,0.000000,0.0
1,2,0,2956,"(0, 2956)",1.0,0.000000,24.0,-0.000088,0.000220,0.000126,...,0.209980,1.185362,-0.112039,-1.079542,-0.279307,-0.799582,0.000000,0.000000,0.000000,0.0
2,3,1,4038,"(1, 4038)",1.0,0.000000,496.0,0.009896,0.000291,0.000891,...,-0.119587,0.369391,0.314860,-0.386489,0.047997,-0.289488,0.000000,0.000000,0.000000,0.0
3,4,2,1848,"(2, 1848)",1.0,0.080000,72.0,0.057307,0.000549,0.000115,...,0.197832,-0.462462,0.697577,0.077155,-0.388575,-1.303094,4.000000,1.912359,0.563811,1.0
4,5,3,513,"(3, 513)",1.0,0.000000,391.0,-0.138788,0.000362,0.000546,...,0.011054,0.265824,0.448829,-0.277887,-0.016421,-0.200409,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,3865,3924,"(3865, 3924)",1.0,0.000000,2.0,-0.000290,0.000071,0.000084,...,0.213741,0.791243,0.508028,-0.492668,0.007899,-0.490364,0.000000,0.000000,0.000000,0.0
1996,1997,3917,4025,"(3917, 4025)",1.0,0.000000,2.0,0.000000,0.000085,0.000265,...,-0.079404,-0.271674,0.390418,-1.787205,-1.797398,-0.456385,0.000000,0.000000,0.000000,0.0
1997,1998,3922,3947,"(3922, 3947)",1.0,0.000000,116.0,0.011228,0.000111,0.000745,...,0.825973,-0.106804,0.103595,-0.554733,-0.897413,-0.051411,0.000000,0.000000,0.000000,0.0
1998,1999,3955,3987,"(3955, 3987)",1.0,0.053333,370.0,0.049551,0.001486,0.000146,...,0.913437,-0.242477,0.552453,-0.058112,-1.184891,-1.153716,22.743486,7.050958,1.183573,1.0


In [58]:
y_test = test["Exist_y"]

In [45]:
X_test = test.drop(columns=["Pair", "Source", "Sink", "Id"])
X_test

,Intercept,JC,PA,KI,PR_s1,PR_s2,n2v_1,n2v_2,n2v_3,n2v_4,...,n2v_14,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20,WCN,WAA,WRA
0,1.0,0.000000,56.0,-0.000552,0.000220,0.000185,-0.071535,-1.156289,0.428712,-0.171515,...,-0.469070,0.123224,0.987519,-0.487011,-1.497914,0.483864,-0.793305,0.000000,0.000000,0.000000
1,1.0,0.000000,24.0,-0.000088,0.000220,0.000126,-0.772605,-0.310315,1.137040,0.307255,...,-0.237035,0.209980,1.185362,-0.112039,-1.079542,-0.279307,-0.799582,0.000000,0.000000,0.000000
2,1.0,0.000000,496.0,0.009896,0.000291,0.000891,0.237227,0.363190,0.767382,1.018122,...,-0.351594,-0.119587,0.369391,0.314860,-0.386489,0.047997,-0.289488,0.000000,0.000000,0.000000
3,1.0,0.080000,72.0,0.057307,0.000549,0.000115,0.038695,-0.783323,0.614125,-0.334246,...,0.268175,0.197832,-0.462462,0.697577,0.077155,-0.388575,-1.303094,4.000000,1.912359,0.563811
4,1.0,0.000000,391.0,-0.138788,0.000362,0.000546,0.614467,-0.302154,0.599535,-0.165801,...,0.226326,0.011054,0.265824,0.448829,-0.277887,-0.016421,-0.200409,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.0,0.000000,2.0,-0.000290,0.000071,0.000084,0.835602,0.010444,0.779621,-0.288281,...,0.440748,0.213741,0.791243,0.508028,-0.492668,0.007899,-0.490364,0.000000,0.000000,0.000000
1996,1.0,0.000000,2.0,0.000000,0.000085,0.000265,0.282751,0.803316,0.753201,0.415312,...,-0.785462,-0.079404,-0.271674,0.390418,-1.787205,-1.797398,-0.456385,0.000000,0.000000,0.000000
1997,1.0,0.000000,116.0,0.011228,0.000111,0.000745,0.436222,-0.442665,0.237506,0.571094,...,-0.823483,0.825973,-0.106804,0.103595,-0.554733,-0.897413,-0.051411,0.000000,0.000000,0.000000
1998,1.0,0.053333,370.0,0.049551,0.001486,0.000146,-0.020466,0.071981,0.411433,-0.274574,...,0.111219,0.913437,-0.242477,0.552453,-0.058112,-1.184891,-1.153716,22.743486,7.050958,1.183573


In [59]:
from scipy.special import expit #the logistic function
sigmoid = expit
Prob = sigmoid(X_test @ w_sklearn)
test["Predicted"] = Prob 
test

,Id,Source,Sink,Pair,Intercept_x,JC_x,PA_x,KI_x,PR_s1_x,PR_s2_x,...,n2v_15,n2v_16,n2v_17,n2v_18,n2v_19,n2v_20,WCN,WAA,WRA,Exist_y
0,1,0,2917,"(0, 2917)",1.0,0.000000,56.0,-0.000552,0.000220,0.000185,...,0.123224,0.987519,-0.487011,-1.497914,0.483864,-0.793305,0.000000,0.000000,0.000000,0.0
1,2,0,2956,"(0, 2956)",1.0,0.000000,24.0,-0.000088,0.000220,0.000126,...,0.209980,1.185362,-0.112039,-1.079542,-0.279307,-0.799582,0.000000,0.000000,0.000000,0.0
2,3,1,4038,"(1, 4038)",1.0,0.000000,496.0,0.009896,0.000291,0.000891,...,-0.119587,0.369391,0.314860,-0.386489,0.047997,-0.289488,0.000000,0.000000,0.000000,0.0
3,4,2,1848,"(2, 1848)",1.0,0.080000,72.0,0.057307,0.000549,0.000115,...,0.197832,-0.462462,0.697577,0.077155,-0.388575,-1.303094,4.000000,1.912359,0.563811,1.0
4,5,3,513,"(3, 513)",1.0,0.000000,391.0,-0.138788,0.000362,0.000546,...,0.011054,0.265824,0.448829,-0.277887,-0.016421,-0.200409,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,3865,3924,"(3865, 3924)",1.0,0.000000,2.0,-0.000290,0.000071,0.000084,...,0.213741,0.791243,0.508028,-0.492668,0.007899,-0.490364,0.000000,0.000000,0.000000,0.0
1996,1997,3917,4025,"(3917, 4025)",1.0,0.000000,2.0,0.000000,0.000085,0.000265,...,-0.079404,-0.271674,0.390418,-1.787205,-1.797398,-0.456385,0.000000,0.000000,0.000000,0.0
1997,1998,3922,3947,"(3922, 3947)",1.0,0.000000,116.0,0.011228,0.000111,0.000745,...,0.825973,-0.106804,0.103595,-0.554733,-0.897413,-0.051411,0.000000,0.000000,0.000000,0.0
1998,1999,3955,3987,"(3955, 3987)",1.0,0.053333,370.0,0.049551,0.001486,0.000146,...,0.913437,-0.242477,0.552453,-0.058112,-1.184891,-1.153716,22.743486,7.050958,1.183573,1.0


In [66]:
Pred = test[["Id", "Predicted"]]
Pred["Predicted"] = Pred["Predicted"].replace(np.nan, 0)
Pred.to_csv('LG_Prediction_with_weak_ties.csv', index=False)
Pred

/Users/abigail/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Id,Predicted
0,1,0.030651
1,2,0.022202
2,3,0.021689
3,4,0.993955
4,5,0.019759
...,...,...
1995,1996,0.028754
1996,1997,0.008089
1997,1998,0.021853
1998,1999,1.000000


In [69]:
y_test = y_test.replace(np.nan, 0)

In [70]:
roc_auc_score(y_test, Pred["Predicted"])

0.9324972601748847

In [33]:
# perform 5 fold cross validation
# X = train[['NCA', 'CN', 'AA', 'RA', 'JC', 'PA', 'KI', 'PR_s1', 'PR_s2']]
# y = train['Exist']

# n_splits=5
# kf = KFold(n_splits=n_splits, shuffle=True)

# clf = LogisticRegression(penalty='none')
# auc = []
# for train_index, test_index in kf.split(X):
    # split the train data and test data
#    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
#    y_train, y_test = y[train_index], y[test_index]
    
#    y_pred = clf.fit(X_train, y_train).predict_proba(X_test)
#    auc.append(roc_auc_score(y_test, y_pred[:,1]))

In [34]:
# y_pred

In [35]:
# print("The average AUC is:", mean(auc))